In [1]:
# https://developer.spotify.com/console/get-audio-features-several-tracks/

In [2]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
import json

# pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
path = "../Resources/Charts CSV/regional-fr-daily-latest.csv"
spotify_data_global = pd.read_csv(path)
spotify_data_global.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Petrouchka (feat. PLK),Soso Maness,406013,https://open.spotify.com/track/03y045SZqaC94yfXl1cU4E
2,2,La kiffance,Naps,391310,https://open.spotify.com/track/1bwhDrXcAtqePp8mNJ0pTe
3,3,Daddy chocolat,Koba LaD,344440,https://open.spotify.com/track/4IbDTrLxImTJfZePAX2nKB
4,4,R9R-LINE (feat. Damso),Laylow,267302,https://open.spotify.com/track/4QoZVMhE66MDCd0igG0hd6


In [4]:
spotify_data_global.to_csv("clean_france.csv", header=False, index=False)

In [5]:
path_france_clean = "clean_france.csv"
spotify_data_france_clean = pd.read_csv(path_france_clean)
spotify_data_france_clean.head()

,Position,Track Name,Artist,Streams,URL
0,1,Petrouchka (feat. PLK),Soso Maness,406013,https://open.spotify.com/track/03y045SZqaC94yfXl1cU4E
1,2,La kiffance,Naps,391310,https://open.spotify.com/track/1bwhDrXcAtqePp8mNJ0pTe
2,3,Daddy chocolat,Koba LaD,344440,https://open.spotify.com/track/4IbDTrLxImTJfZePAX2nKB
3,4,R9R-LINE (feat. Damso),Laylow,267302,https://open.spotify.com/track/4QoZVMhE66MDCd0igG0hd6
4,5,Beggin',Måneskin,266547,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e


In [6]:
spotify_data_france_clean["ID"] = spotify_data_france_clean["URL"].str[31:]
spotify_data_france_clean.head()

,Position,Track Name,Artist,Streams,URL,ID
0,1,Petrouchka (feat. PLK),Soso Maness,406013,https://open.spotify.com/track/03y045SZqaC94yfXl1cU4E,03y045SZqaC94yfXl1cU4E
1,2,La kiffance,Naps,391310,https://open.spotify.com/track/1bwhDrXcAtqePp8mNJ0pTe,1bwhDrXcAtqePp8mNJ0pTe
2,3,Daddy chocolat,Koba LaD,344440,https://open.spotify.com/track/4IbDTrLxImTJfZePAX2nKB,4IbDTrLxImTJfZePAX2nKB
3,4,R9R-LINE (feat. Damso),Laylow,267302,https://open.spotify.com/track/4QoZVMhE66MDCd0igG0hd6,4QoZVMhE66MDCd0igG0hd6
4,5,Beggin',Måneskin,266547,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e,3Wrjm47oTz2sjIgck11l5e


In [7]:
len("https://open.spotify.com/track/")

31

In [8]:
test = spotify_data_france_clean["URL"].str
print(test[31:])

0      03y045SZqaC94yfXl1cU4E
1      1bwhDrXcAtqePp8mNJ0pTe
2      4IbDTrLxImTJfZePAX2nKB
3      4QoZVMhE66MDCd0igG0hd6
4      3Wrjm47oTz2sjIgck11l5e
                ...          
195    2FHsE83xbrzzWusudCFpg1
196    1tkg4EHVoqnhR6iFEXb60y
197    3QNjxtei6jekNeNJzRbwI6
198    0sydMl87Twrbrc5CS5Xssi
199    0LThjFY2iTtNdd4wviwVV2
Name: URL, Length: 200, dtype: object


In [ ]:
token = "BQDkUVHSHlnDbARSqmu8syVDQGhqAeZee8NYPICTtocLBkqD3N-h6ziZHUQVFVyi3N6wtI1lKgmlkwKB2nqq0GTkkDTxNkg1GdyrrUkCbe1g9x4UyaCT2vILcmGU5VzIS9QFcMHqFPJ0aqAyNKb1zcRFwwYgzcoUoLdvIdF1xLjQUYcub8Ey7UrahpmoWr5C7zhnkqV0m6Z4T7dmFoIARROaB8AzsxJD2tJHVef24fMmmice1OxfzHedCoi_KaTkTlT9-E5VZHfOKwGuR76T"
headers_dict = {"Authorization": f'Bearer {token}'}

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.SpotifyTopChartsdb
collection = db.TopFrance
collection.drop()

rows=len(spotify_data_france_clean)
global_dict={}
# spotify_artist_dict=[]

for row in range(0,rows):
    # Hacer el llamado a la API de Spotify usando spotify_data_wNA.iloc[row,4]
    artist_id = spotify_data_france_clean.iloc[row,5]
    artist_name = spotify_data_france_clean.iloc[row,2]
    track_name = spotify_data_france_clean.iloc[row,1]

    
    response = requests.get(f'https://api.spotify.com/v1/audio-features?ids={artist_id}', headers=headers_dict).json()
    # print(response)
    # artist_id = response["audio_features"]
    # print(response["audio_features"])

    danceability = response["audio_features"][0]["danceability"]
    energy = response["audio_features"][0]["energy"]
    key = response["audio_features"][0]["liveness"]
    loudness = response["audio_features"][0]["key"]
    acousticness = response["audio_features"][0]["acousticness"]
    instrumentalness = response["audio_features"][0]["instrumentalness"]
    liveness = response["audio_features"][0]["liveness"]
    valence = response["audio_features"][0]["valence"]
    tempo = response["audio_features"][0]["tempo"]
    track_href = response["audio_features"][0]["track_href"]
        
        
    global_dict={'Artist Name':artist_name,
                 'Track Name':track_name,
                 'Danceability': danceability,
                 'Energy': energy,
                 'Key': key,
                 'Loudness': loudness,
                 'Acousticness': acousticness,
                 'Instrumentalness': instrumentalness,
                 'Liveness': liveness,
                 'Valence': valence,
                 'Tempo': tempo,
                 'Track Reference': track_href
                 }
    collection.insert_one(global_dict)
#     spotify_artist_dict.append(artist_dict)
   
    print(global_dict)

{'Artist Name': 'Soso Maness', 'Track Name': 'Petrouchka (feat. PLK)', 'Danceability': 0.624, 'Energy': 0.802, 'Key': 0.252, 'Loudness': 6, 'Acousticness': 0.0597, 'Instrumentalness': 0, 'Liveness': 0.252, 'Valence': 0.818, 'Tempo': 133.888, 'Track Reference': 'https://api.spotify.com/v1/tracks/03y045SZqaC94yfXl1cU4E', '_id': ObjectId('60f790bb4861b9cdf8200254')}
{'Artist Name': 'Naps', 'Track Name': 'La kiffance', 'Danceability': 0.805, 'Energy': 0.637, 'Key': 0.071, 'Loudness': 6, 'Acousticness': 0.228, 'Instrumentalness': 0, 'Liveness': 0.071, 'Valence': 0.678, 'Tempo': 140.045, 'Track Reference': 'https://api.spotify.com/v1/tracks/1bwhDrXcAtqePp8mNJ0pTe', '_id': ObjectId('60f790bc4861b9cdf8200255')}
{'Artist Name': 'Koba LaD', 'Track Name': 'Daddy chocolat', 'Danceability': 0.721, 'Energy': 0.523, 'Key': 0.281, 'Loudness': 7, 'Acousticness': 0.0693, 'Instrumentalness': 0, 'Liveness': 0.281, 'Valence': 0.757, 'Tempo': 141.034, 'Track Reference': 'https://api.spotify.com/v1/tracks/4I

{'Artist Name': 'MHD', 'Track Name': 'Pololo', 'Danceability': 0.773, 'Energy': 0.776, 'Key': 0.0571, 'Loudness': 1, 'Acousticness': 0.316, 'Instrumentalness': 7.15e-05, 'Liveness': 0.0571, 'Valence': 0.895, 'Tempo': 106.022, 'Track Reference': 'https://api.spotify.com/v1/tracks/2gmTz7pHMqAk6fpOc88qxA', '_id': ObjectId('60f790c14861b9cdf820026c')}
{'Artist Name': 'Laylow', 'Track Name': 'UN RÊVE ÉTRANGE', 'Danceability': 0.652, 'Energy': 0.516, 'Key': 0.15, 'Loudness': 4, 'Acousticness': 0.125, 'Instrumentalness': 0, 'Liveness': 0.15, 'Valence': 0.502, 'Tempo': 76.544, 'Track Reference': 'https://api.spotify.com/v1/tracks/0L1JUpc6bjrBvK9pqMTGcb', '_id': ObjectId('60f790c14861b9cdf820026d')}
{'Artist Name': 'Joé Dwèt Filé', 'Track Name': 'Jolie madame (feat. Ronisia)', 'Danceability': 0.756, 'Energy': 0.563, 'Key': 0.138, 'Loudness': 1, 'Acousticness': 0.276, 'Instrumentalness': 1.02e-06, 'Liveness': 0.138, 'Valence': 0.88, 'Tempo': 159.985, 'Track Reference': 'https://api.spotify.com/v

{'Artist Name': 'Justin Bieber', 'Track Name': 'Peaches (feat. Daniel Caesar & Giveon)', 'Danceability': 0.677, 'Energy': 0.696, 'Key': 0.42, 'Loudness': 0, 'Acousticness': 0.321, 'Instrumentalness': 0, 'Liveness': 0.42, 'Valence': 0.464, 'Tempo': 90.03, 'Track Reference': 'https://api.spotify.com/v1/tracks/4iJyoBOLtHqaGxP12qzhQI', '_id': ObjectId('60f790c54861b9cdf8200284')}
{'Artist Name': 'Laylow', 'Track Name': 'QUE LA PLUIE', 'Danceability': 0.536, 'Energy': 0.393, 'Key': 0.118, 'Loudness': 11, 'Acousticness': 0.436, 'Instrumentalness': 0.000877, 'Liveness': 0.118, 'Valence': 0.21, 'Tempo': 85.115, 'Track Reference': 'https://api.spotify.com/v1/tracks/5WxF9OZW6Fn5AN26J3RkWW', '_id': ObjectId('60f790c64861b9cdf8200285')}
{'Artist Name': 'Imagine Dragons', 'Track Name': 'Follow You', 'Danceability': 0.542, 'Energy': 0.732, 'Key': 0.496, 'Loudness': 9, 'Acousticness': 0.00209, 'Instrumentalness': 7.92e-05, 'Liveness': 0.496, 'Valence': 0.489, 'Tempo': 124.912, 'Track Reference': 'htt